In [1]:
from topagnps2cche1d.network import Node, Reach, System

In [2]:
s = System()

In [3]:
reach1 = Reach(id=1)
reach2 = Reach(id=2, receiving_reach_id=1)
reach3 = Reach(id=3, receiving_reach_id=2)
reach4 = Reach(id=4, receiving_reach_id=2)
reach5 = Reach(id=5, receiving_reach_id=1)
reach6 = Reach(id=6, receiving_reach_id=3)

In [4]:
s.add_reach(reach1)
s.add_reach(reach2)
s.add_reach(reach3)
s.add_reach(reach4)
s.add_reach(reach5)
s.add_reach(reach6)

In [5]:
# reach2.include_reach()
reach2.ignore_reach()

In [6]:
s.assess_connectivity()
s.connectivity_dict

{2: [3, 4], 1: [5], 3: [6]}

In [7]:
s.assign_strahler_number_to_reaches()

In [8]:
reach6.strahler_number

In [9]:
s.find_root_reach()

1